# libs & config

In [1]:
# ajustando o caminho que o python vai procurar os arquivos
import os
import sys

# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

# PIPE

## 1 - Coleta e Identificação da Origem dos Dados (extract)

### 1.1 Recebimento do arquivo de origem & 1.2 Classificação da Planilha (Tipo e Finalidade)

In [7]:
from Scripts.Extract import ReadArchive, ConvertXml_toExcel, GetFile
# checa a apsta e se achar um arquivo, pega o path extençaõ para usar, SE NAO TIVER AS EXTENSAO ACEITA RETORNA ERRO
# ja esta fazendo o 2.3

found_excel_path = GetFile('xlsx')

if found_excel_path:  # vai direto
    file_path = found_excel_path
    df_prod = ReadArchive(file_path, 1)
else:
    found_excel_path = GetFile('xml')
    if found_excel_path:  # converte e depois le
        file_path = ConvertXml_toExcel(found_excel_path)
        df_prod = ReadArchive(file_path, 0)




Arquivo encontrado: d:\kaue\projetos\pipeline_estoque_bling_masquerade\Data\ESTOQUE MASUQUERADE.XLSX


## 2. Classificação da Planilha (Tipo e Finalidade)


### 2.1 Tipo de produto & 2.2 Tipo de movimentação

## 4. 4. Transformação e Validação (Transform)
### 4.1 Carregar produtos já existentes no Bling via API

In [4]:
from Scripts.Extract import GetProdutos, GetToken

token = GetToken() # precisa estar no arquivo .env
df_prod_bling = GetProdutos(token) # ja devolve em um DataFrame



Página 2 obtida com sucesso, total de produtos: 100
Página 3 obtida com sucesso, total de produtos: 100
Página 4 obtida com sucesso, total de produtos: 100
Página 5 obtida com sucesso, total de produtos: 100
Página 6 obtida com sucesso, total de produtos: 100
Página 7 obtida com sucesso, total de produtos: 100
Página 8 obtida com sucesso, total de produtos: 100
Página 9 obtida com sucesso, total de produtos: 100
Página 10 obtida com sucesso, total de produtos: 100
Página 11 obtida com sucesso, total de produtos: 100
Página 12 obtida com sucesso, total de produtos: 100
Página 13 obtida com sucesso, total de produtos: 100
Página 14 obtida com sucesso, total de produtos: 100
Página 15 obtida com sucesso, total de produtos: 100
Página 16 obtida com sucesso, total de produtos: 100
Página 17 obtida com sucesso, total de produtos: 100
Página 18 obtida com sucesso, total de produtos: 100
Página 19 obtida com sucesso, total de produtos: 100
Página 20 obtida com sucesso, total de produtos: 100
P

In [ ]:
df_prod

# --  função PadronizeDataFrame
# 1. limpe os que nao fomos usar vazio e criar a coluna grade
# 2. transforme as colunas de tamnho em chaves de um diconario, apenas coluans que tiverem valor, adione na coluna grade
# 3. adicone os valores no diconario com base da scolunas que esta no  diconario, apois adicona na grade apaga as outras colunas
# 4. crie um produto(deve seguir o padro bling "cod,cor[setiver],tam") para cada chave da grade,e crie a coluna quant aond eela receb o respectivo valor da chave
# 5. veja quais produtos nao bate com o da bling, e crie uma coluna de status, se esta ok(true) ou nao(false)
# 6. compare com os produtos da bling, concertando codigos errados atravez de quanto for parecido o nome e cod se concertar atribui ok
# 7. compare novamente , se haver ainda dierentes cria um novo df para criar os produtos que nao existem na bling





,CÓDIGO,INFANTIL MENINOS,U,PP,P,M,G,GG,total:
0,ED 329,Capa Manto Akatsuki,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ED 329 A,Capa Manto c/Forro,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HW017,Capa Infantil Preta Bember 3/6 anos P,NaN,NaN,15.0,NaN,NaN,NaN,NaN
3,HW 018,Capa Infantil Preta Bember 8/12 anos M,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,HW 115,Capa Cetim c/ Gola (Preta / Vermelha),NaN,NaN,15.0,17.0,86.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
133,HW 578,Tiara Halloween Diabo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,HW 579,Tiara Monstrinho (LANÇAMENTO),NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,HW 580,Luva Esqueleto,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,NaN,bolsa abobora,462.0,NaN,NaN,NaN,NaN,NaN,NaN
